In [3]:
import numpy as numpy
import pandas as pd 
from fuzzywuzzy import process
import pycountry
import re
import numpy as np

In [5]:
#Load dataset
airplane=pd.read_csv("../Data/Airplane_Crashes_and_Fatalities_Since_1908.csv")

In [6]:
# Display the first five rows of dataset
airplane.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [7]:
airplane.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          5268 non-null   object 
 1   Time          3049 non-null   object 
 2   Location      5248 non-null   object 
 3   Operator      5250 non-null   object 
 4   Flight #      1069 non-null   object 
 5   Route         3562 non-null   object 
 6   Type          5241 non-null   object 
 7   Registration  4933 non-null   object 
 8   cn/In         4040 non-null   object 
 9   Aboard        5246 non-null   float64
 10  Fatalities    5256 non-null   float64
 11  Ground        5246 non-null   float64
 12  Summary       4878 non-null   object 
dtypes: float64(3), object(10)
memory usage: 535.2+ KB


#### 1. Addressing missing value:

In [8]:
#Numeric values 
num_columns=airplane.select_dtypes('float64').columns
for col in num_columns:
    airplane[col].fillna(airplane[col].mean(),inplace=True)
#Drop the columns that have more than 50% of missing values :
airplane.drop(columns="Flight #", inplace=True)

#Fill Summary columns with `No remark` 
airplane["Summary"]=airplane['Summary'].replace({pd.NA: 'No remark', None: 'No remark'})

#Fill Registration column with `unavailabele`
airplane['Registration']=airplane["Registration"].fillna('unavailable')

#Fill  column with `unavailabele`
airplane['Route']=airplane["Route"].fillna('unavailable')

#Fill ojbect columns with mode 
col_obj=["Location","Operator","Type","cn/In"]
for col in col_obj:
    airplane[col].fillna(airplane[col].mode()[0],inplace=True)

#### 2.Correcting Data types :

In [9]:
# Convert date columns to datetime
airplane['Date'] = pd.to_datetime(airplane['Date'], errors='coerce')

#Convert Aboard , Fatalities and Ground to integer dtype :
for col in num_columns:
    airplane[col]=airplane[col].astype(int)

# Convert categorical columns to category dtype
categorical_cols = [ 'Location', 'Operator', 'Type', 'Route']
for col in categorical_cols:
    airplane[col] = airplane[col].astype('category')

#### 3.Removing Duplicates :

In [10]:
# Check for duplicates
duplicates = airplane.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


#### 4.Remove useless Columns :


In [11]:
#Create list of useless columns
columns_to_drop = ['cn/In']
columns_to_drop_existing = [col for col in columns_to_drop if col in airplane.columns]

#Drop useless columns:
if columns_to_drop_existing:
    airplane = airplane.drop(columns=columns_to_drop_existing, errors='ignore')


#### 5.Create New Features:

In [12]:
# Extract Year
airplane['Year'] = airplane['Date'].dt.year
# Extract Month 
# Mapping numeric months to their respective names
month_map = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 
             6: 'June', 7: 'July', 8: 'August', 9: 'September', 
             10: 'October', 11: 'November', 12: 'December'}

# Replace numeric months with names
airplane['Month'] = airplane['Date'].dt.month
airplane['Month'] = airplane['Month'].map(month_map)

# Now you can use the earlier method with month names
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

airplane['Month'] = pd.Categorical(airplane['Month'], categories=month_order, ordered=True)
airplane['Day'] = airplane['Date'].dt.day_name().astype('category')

In [13]:
# Create a list of country names for matching
country_list = [country.name for country in pycountry.countries]

# US state full names
us_state_full_name = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
                     "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
                     "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska",
                     "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio",
                     "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee",
                     "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

# US state abbreviations
us_state_abbr_name = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY",
                     "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND",
                     "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

def extract_country(location_text):
    """
    Extract and correct country name from a free-text location string.
    If a US state is found, return 'USA'.
    """
    if pd.isna(location_text):
        return None
    
    # First check if location contains a US state (full name or abbreviation)
    location_upper = location_text.upper()
    words = re.findall(r"[A-Za-z\s]+", location_text)
    
    # Check for US state full names (case-insensitive)
    for state in us_state_full_name:
        if state.upper() in location_upper:
            return 'USA'
    
    # Check for US state abbreviations (as whole words)
    location_words = re.findall(r"\b[A-Za-z]+\b", location_text)
    for word in location_words:
        if word.upper() in us_state_abbr_name:
            return 'USA'
    
    # If no US state found, proceed with country matching
    words = re.findall(r"[A-Za-z]+", location_text)
    best_match = None
    best_score = 0
    
    for word in words:
        match, score = process.extractOne(word, country_list)
        if score > best_score:
            best_match = match
            best_score = score
    
    if best_score >= 85:
        return best_match
    else:
        return None

# Apply country extraction to the Location column
airplane['Country'] = airplane['Location'].apply(extract_country)

In [14]:
#Create the Operator type ( Military Or Civilian Operator ):
Military=['Military','Navy','Army']
condition=airplane["Operator"].str.contains("Military|Navy|Army")
airplane["Operator_Type"]=np.select([condition],['Military'],default="Civilian")

#Convert Operator_type to category :
airplane["Operator_Type"]=airplane["Operator_Type"].astype('category')

In [15]:
# Create Fatality rate
airplane['Fatality_Rate'] = (airplane['Fatalities'] / airplane['Aboard']).round(2)

# Handle infinite or NaN values created by division
airplane['Fatality_Rate'] = airplane['Fatality_Rate'].replace([np.inf, -np.inf], np.nan)
airplane.drop(airplane[airplane['Fatality_Rate'] > 1].index, inplace=True)
airplane['Fatality_Rate'] = airplane['Fatality_Rate'].fillna(0)

In [28]:
# Define cause categories with keywords
cause_keywords = {
    'Pilot Error': ['pilot', 'altitude', 'approach', 'landing', 'takeoff'],
    'Mechanical Failure': ['engine', 'failure', 'control', 'mechanical'],
    'Weather': ['weather', 'storm', 'conditions', 'lightning', 'turbulence'],
    'Collision/Impact': ['crashed', 'struck', 'mountain', 'obstacle', 'building'],
    'Fire/Explosion': ['fire', 'explosion', 'burned'],
    'Runway Issues': ['runway', 'overshot', 'undershot', 'airport'],
    'Crew Error': ['crew', 'fatigue', 'miscommunication', 'mistake', 'error'],
    'Hijacking/Terrorism': ['hijack', 'bomb', 'terrorist', 'explosive', 'attack'],
    'Military Action': ['shot', 'missile', 'military', 'combat', 'war'],
    'Maintenance Issues': ['maintenance', 'inspected', 'repair', 'servicing'],
    'Bird Strike': ['bird', 'flock', 'ingested'],
    'Sabotage': ['sabotage', 'intentional', 'deliberate']
}

# Extract only the first cause found
def extract_cause(summary):
    summary = str(summary).lower()
    for cause, keywords in cause_keywords.items():
        if any(keyword in summary for keyword in keywords):
            return cause
    return 'Unknown'

# Apply to summary column - returns single string, no brackets
airplane['Causes'] = airplane['Summary'].apply(extract_cause)



#### 6. Correcting Inaccurate Data:

In [18]:
#Extract unique hour and minutes values :
print("The unique values in the Hour partition :\n", airplane["Time"].str.split(":",expand=True)[0].unique())
print("The unique values in the Minute partition :\n", airplane["Time"].str.split(":",expand=True)[1].unique())

The unique values in the Hour partition :
 ['17' '06' nan '18' '10' '01' '15' '23' '05' '08' '07' '21' '02' '13' '09'
 'c' '22' '20' '04' '14' '12' '00' '03' '19' '11' '16' '1' 'c16' "12'20"
 '18.40' '114' 'c14' '0943' '2' "22'08" '8' '9']
The unique values in the Minute partition :
 ['18' '30' nan '00' '20' '45' '15' '40' ' 1' '23' '17' '05' ' 2' '48' '09'
 '07' '35' '42' '52' '36' '08' '19' '32' '10' '47' '14' '38' '50' '25'
 '56' '11' '41' '51' '13' '04' '27' '22' '43' '54' '16' '58' '26' '55'
 '49' '59' '03' '12' '31' '33' '57' '39' '29' '44' '01' '34' '53' '28'
 '46' '37' '24' '02' '21' '06' None ' 9']


In [19]:
error_m=[' 1',' 2',None,' 9']# list of minute error 
error_h=['c','1','c16',"12'20","18.40",'114','c14','0943','2',"22'08",'8','9'] #list of hour errors
Time_error_1 = airplane.loc[airplane["Time"].str.split(":",expand=True)[0].isin(error_h), 'Time']
Time_error_2 = airplane.loc[airplane["Time"].str.split(":",expand=True)[1].isin(error_m), 'Time']
#extract subset of time erorrs
Time_error=pd.concat([Time_error_1,Time_error_2])
#display result 
print("List of Time errors : \n",Time_error.values)

List of Time errors : 
 ['c: 1:00' 'c:17:00' 'c: 2:00' 'c:09:00' '1:30' 'c16:50' "12'20" '18.40'
 'c:09:00' '114:20' 'c14:30' '0943' '1:00' '2:40' "22'08" 'c: 9:40' '2:00'
 '8:02' '9:30' 'c: 1:00' 'c: 2:00' "12'20" '18.40' '0943' "22'08"
 'c: 9:40']


In [20]:
#the correct values 
correct_time= ['1:00' ,'17:00', '02:00','09:00', '01:30' ,'16:50', '12:20','18:40','09:00',
               '14:20', '14:30','09:43', '01:00' ,'02:40', "22:08",'09:40','02:00','08:02',
               '09:30','01:00' ,'02:00', "12:20" ,'18:40', '09:43', '22:08','09:40']
the_index=Time_error.index

# Ensure the lists are of the same length :
if len(correct_time) == len(the_index):
    # Replace values at the specified indices :
    airplane.loc[the_index, 'Time'] = correct_time
else:
    print("The length of 'correct_time' and 'the_index' must be the same.")
    
# Convert 'Time' column to datetime and format it to hh:mm :
airplane['Time'] = pd.to_datetime(airplane['Time'], format='%H:%M').dt.strftime('%H:%M')

#Fill Time missing value
airplane['Time']=airplane['Time'].fillna(method="ffill")

In [21]:
# Standardize text data
categorical_cols=airplane.select_dtypes("category").columns
for col in categorical_cols:
    airplane[col] = airplane[col].str.title().str.strip()

#### 7. Final dataset: 

In [29]:
order=['Date','Year','Month','Day','Time','Country','Location','Operator','Operator_Type', 'Route', 'Type', 'Aboard',
       'Fatalities', 'Fatality_Rate','Ground','Causes' ,'Summary']
airplane=airplane[order]
airplane.head()

,Date,Year,Month,Day,Time,Country,Location,Operator,Operator_Type,Route,Type,Aboard,Fatalities,Fatality_Rate,Ground,Causes,Summary
0,1908-09-17,1908,September,Thursday,17:18,USA,"Fort Myer, Virginia",Military - U.S. Army,Military,Demonstration,Wright Flyer Iii,2,1,0.5,0,Mechanical Failure,"During a demonstration flight, a U.S. Army fly..."
1,1912-07-12,1912,July,Friday,06:30,USA,"Atlanticity, New Jersey",Military - U.S. Navy,Military,Test Flight,Dirigible,5,5,1.0,0,Pilot Error,First U.S. dirigible Akron exploded just offsh...
2,1913-08-06,1913,August,Wednesday,06:30,Canada,"Victoria, British Columbia, Canada",Private,Civilian,Unavailable,Curtiss Seaplane,1,1,1.0,0,Weather,The first fatal airplane accident in Canada oc...
3,1913-09-09,1913,September,Tuesday,18:30,Northern Mariana Islands,Over The North Sea,Military - German Navy,Military,Unavailable,Zeppelin L-1 (Airship),20,14,0.7,0,Mechanical Failure,The airship flew into a thunderstorm and encou...
4,1913-10-17,1913,October,Friday,10:30,Germany,"Near Johannisthal, Germany",Military - German Navy,Military,Unavailable,Zeppelin L-2 (Airship),30,30,1.0,0,Mechanical Failure,Hydrogen gas which was being vented was sucked...


In [30]:
airplane.to_csv("../Data/cleaned_airplane_crashes.csv", index=False)

<div style="border-radius:10px; border: #dc7633  solid; padding: 20px; background-color:#f6ddcc
            ; font-size:110%; text-align:left">

 <span style="color: #873600 "> **Summary**: </br>

1. <span style="color:#a93226">Handling Missing Values :</span></br>

   <span style="color: #17202a"> Visualized missing data using a horizontal bar chart and identified columns with more than 50% missing values."</br>
   <span style="color: #17202a"> Filled missing values in the 'Summary' column with 'No remark'."</br>
   <span style="color: #17202a"> Filled missing values in the 'Location','Operator', 'Type' and 'cn/In' columns with mode variable."</br>
   <span style="color: #17202a"> Filled missing values in the 'Registration' and 'Route' columns with 'No remark'."</br>
    <span style="color: #17202a">Filled numerical columns with mean values.</br>
   <span style="color: #17202a">Dropped 'Fligh # ' column due to high missing values.</br>

2. <span style="color:#a93226"> Correcting Data types :  </span>

   <span style="color: #17202a"> Converted 'Date' to datetime.</br>
   <span style="color: #17202a"> Converted categorical columns to category dtype.</br>
   <span style="color: #17202a"> Converted numerical  columns to integer dtype</br>
3. <span style="color:#a93226"> Removing Duplicates : 

   <span style="color: #17202a"> Checked for duplicate rows and did not find any.
4. <span style="color:#a93226"> Remove useless Columns :</span> 

    <span style="color: #17202a"> Removing 'cn/In' column. </br>
5. <span style="color:#a93226"> Create New Features:</span>  

   <span style="color: #17202a">  Extract Year, Month, and Day name from the Date column.</br>
   <span style="color: #17202a"> Extract Country from the Location column and correct country errors (misspellings, convert US states to USA). </br>
   <span style="color: #17202a"> Extract Operator_Type from Operator, divide into 'Military' and 'Civilian'.</br>
   <span style="color: #17202a">  Calculate Fatality_Rate, the percentage of fatalities.</br>
   
6. <span style="color:#a93226"> Correcting Inaccurate Data: </span>

    <span style="color: #17202a"> Extract and Correct the Time errors. </br>
    <span style="color: #17202a"> Convert the categorical columns to Title form and remove white space.
